In [16]:
"""
The MIT License (MIT)

Copyright © 2023 Dr Keith S Reid Cailleach Computing Ltd

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and 
associated documentation files (the “Software”), to deal in the Software without restriction, 
including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial
portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

=#

#=

Python version
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux

Date 19 Feb 2023

Box Spec

            .-/+oossssoo+/-.               
        `:+ssssssssssssssssss+:`           ------------ 
      -+ssssssssssssssssssyyssss+-         OS: Ubuntu 22.04.1 LTS x86_64 
    .ossssssssssssssssssdMMMNysssso.       Host: 
   /ssssssssssshdmmNNmmyNMMMMhssssss/      Kernel: 5.15.0-60-generic 
  +ssssssssshmydMMMMMMMNddddyssssssss+     Uptime: 
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/    Packages: 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Shell: bash 5.1.16 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   Resolution: 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   DE: GNOME 
ossyNMMMNyMMhsssssssssssssshmmmhssssssso   WM: Mutter 
+sssshhhyNMMNyssssssssssssyNMMMysssssss+   WM Theme: Adwaita 
.ssssssssdMMMNhsssssssssshNMMMdssssssss.   Theme: 
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/    Icons: 
  +sssssssssdmydMMMMMMMMddddyssssssss+     Terminal: 
   /ssssssssssshdmNNNNmyNMMMMhssssss/      CPU: AMD Ryzen 9 3900X (24) @ 3.800G 
    .ossssssssssssssssssdMMMNysssso.       GPU: 
      -+sssssssssssssssssyyyssss+-         Memory: 64Gb RAM
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.                                       
                                                                   

=#

#=
Intent:
I love Python and Julia
Julia is faster for some stuff
Implement Stern Brocot in Python and Julia using TDD and nothing too clever
For speed comparison
Roughly simlar logic in the two versions
=#

"""

# 1 Libraries

import numpy as np
import timeit

# 2 Classes

class RationalNode:
    def __init__(self, numerator, denominator, place, layer):
        self.numerator      = numerator
        self.denominator    = denominator
        self.place          = place
        self.layer          = layer

# 3 Config

def get_depth():
    return 1

# 4 model

def count_nodes(depth):
    node_count = (2**depth)-1 # Mersene, n'est pas? 
    return node_count

def build_tree():
    # tree has configurable depth see config
    depth           = get_depth()
    node_count      = count_nodes(depth)
    powers          = []
    over_power      = 2**depth
    this_power      = int(over_power/2)
    while this_power >= 1:
        powers.append(this_power)
        this_power = int(this_power/2)
    for current_layer in range(depth):
        this_power      = powers[current_layer]
        if current_layer == 0:
            # these predate root and are loop invariants
            zero_over_one   = RationalNode(0,1,0,           current_layer)
            one_over_zero   = RationalNode(1,0,node_count+1,current_layer)
            triangular_part = [RationalNode(None,None,i+1,None) for i in range(node_count)]    
            tree            = [zero_over_one]
            tree.extend(triangular_part)
            tree.extend([one_over_zero])
            # child logic
            left_parent         = tree[this_power+this_power]
            right_parent        = tree[this_power-this_power]
            child_place         = this_power
            child_numerator     = left_parent.numerator   + right_parent.numerator
            child_denominator   = left_parent.denominator + right_parent.denominator
            child               = RationalNode(child_numerator,child_denominator,child_place,current_layer+1)
            tree[child.place]   = child
            child_places        = [child.place]
        else:
            grand_child_places  = []
            for parent_place in child_places:
                left_child_place    =  parent_place - this_power 
                right_child_place   =  parent_place + this_power
                grand_child_places.extend([left_child_place, right_child_place])
            child_places = grand_child_places
            for this_child_place in child_places:
                left_parent         = tree[this_child_place+this_power]
                right_parent        = tree[this_child_place-this_power]
                child_place         = this_power
                child_numerator     = left_parent.numerator   + right_parent.numerator
                child_denominator   = left_parent.denominator + right_parent.denominator
                child               = RationalNode(child_numerator,child_denominator,this_child_place,current_layer+1)
                tree[child.place]   = child
    return tree

# 5 view

def draw_tree(tree):
    depth       = get_depth()
    over_power  = 2**depth
    height      = int(1+depth)
    width       = int(1+over_power)
    numerator_host   = np.zeros((height,width))
    denominator_host = np.zeros((height,width))

    for node in tree:
        numerator_host[node.layer][node.place]  = int(node.numerator)
        denominator_host[node.layer][node.place]= int(node.denominator)

    for num_row in enumerate(numerator_host):
        both = zip(num_row[1],denominator_host[num_row[0]])
        num_printable = ""
        den_printable = ""
        for x in both:
            if x == (0.0,0.0):
                num_printable = num_printable+(" ")
                den_printable = den_printable+(" ")
            else:
                num_printable = num_printable+(str(int(x[0])))
                den_printable = den_printable+(str(int(x[1])))
        print(num_printable)
        print(den_printable)
    

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    
    draw_tree(tree)

    print("\n")
    
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
                    
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of", depth, "layers.")


0 1
1 0
 1 
 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 1
place: 2 	frac:	 1 / 0

Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  2.879590399970766e-06  seconds.
At a depth of 1 layers.


In [17]:

# 3 Config

def get_depth():
    return 2

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    draw_tree(tree)
    print("\n")
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of", depth, "layers.")


0   1
1   0
  1  
  1  
 1 2 
 2 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 2
place: 2 	frac:	 1 / 1
place: 3 	frac:	 2 / 1
place: 4 	frac:	 1 / 0

Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  5.017571400003362e-06  seconds.
At a depth of 2 layers.


In [18]:

# 3 Config

def get_depth():
    return 4

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    draw_tree(tree)
    print("\n")
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nMy Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of", depth, "layers.")

0               1
1               0
        1        
        1        
    1       2    
    2       1    
  1   2   3   3  
  3   3   2   1  
 1 2 3 3 4 5 5 4 
 4 5 5 4 3 3 2 1 


place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 4
place: 2 	frac:	 1 / 3
place: 3 	frac:	 2 / 5
place: 4 	frac:	 1 / 2
place: 5 	frac:	 3 / 5
place: 6 	frac:	 2 / 3
place: 7 	frac:	 3 / 4
place: 8 	frac:	 1 / 1
place: 9 	frac:	 4 / 3
place: 10 	frac:	 3 / 2
place: 11 	frac:	 5 / 3
place: 12 	frac:	 2 / 1
place: 13 	frac:	 5 / 2
place: 14 	frac:	 3 / 1
place: 15 	frac:	 4 / 1
place: 16 	frac:	 1 / 0

My Python Stern-Brocot
implementation takes  1.4577579199976754e-05  seconds.
At a depth of 4 layers.


In [19]:

# 3 Config

def get_depth():
    return 8

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    #draw_tree(tree)
    print("\n")
    for i in tree:
        print("place:",i.place,"\tfrac:\t",i.numerator,"/",i.denominator)
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of", depth, "layers.")




place: 0 	frac:	 0 / 1
place: 1 	frac:	 1 / 8
place: 2 	frac:	 1 / 7
place: 3 	frac:	 2 / 13
place: 4 	frac:	 1 / 6
place: 5 	frac:	 3 / 17
place: 6 	frac:	 2 / 11
place: 7 	frac:	 3 / 16
place: 8 	frac:	 1 / 5
place: 9 	frac:	 4 / 19
place: 10 	frac:	 3 / 14
place: 11 	frac:	 5 / 23
place: 12 	frac:	 2 / 9
place: 13 	frac:	 5 / 22
place: 14 	frac:	 3 / 13
place: 15 	frac:	 4 / 17
place: 16 	frac:	 1 / 4
place: 17 	frac:	 5 / 19
place: 18 	frac:	 4 / 15
place: 19 	frac:	 7 / 26
place: 20 	frac:	 3 / 11
place: 21 	frac:	 8 / 29
place: 22 	frac:	 5 / 18
place: 23 	frac:	 7 / 25
place: 24 	frac:	 2 / 7
place: 25 	frac:	 7 / 24
place: 26 	frac:	 5 / 17
place: 27 	frac:	 8 / 27
place: 28 	frac:	 3 / 10
place: 29 	frac:	 7 / 23
place: 30 	frac:	 4 / 13
place: 31 	frac:	 5 / 16
place: 32 	frac:	 1 / 3
place: 33 	frac:	 6 / 17
place: 34 	frac:	 5 / 14
place: 35 	frac:	 9 / 25
place: 36 	frac:	 4 / 11
place: 37 	frac:	 11 / 30
place: 38 	frac:	 7 / 19
place: 39 	frac:	 10 / 27
place: 40 	frac

In [20]:

# 3 Config

def get_depth():
    return 12

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of", depth, "layers.")



Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  0.0033494253342999402  seconds.
At a depth of 12 layers.


In [21]:

# 3 Config

def get_depth():
    return 13

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of", depth, "layers.")



Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  0.0067446708239000145  seconds.
At a depth of 13 layers.


In [23]:

# 3 Config

def get_depth():
    return 14

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of", depth, "layers.")



Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  0.013931729978600016  seconds.
At a depth of 14 layers.


In [24]:

# 3 Config

def get_depth():
    return 15

# 6 control

if __name__=="__main__":
    tree        = build_tree()
    myriad          = 10000
    depth           = get_depth()
    average_speed   = timeit.timeit(build_tree, number=myriad)/myriad
    print("\nAveraged over a myriad of repeats my Python Stern-Brocot\nimplementation takes ", average_speed, " seconds.")
    print("At a depth of", depth, "layers.")



Averaged over a myriad of repeats my Python Stern-Brocot
implementation takes  0.030198445934199937  seconds.
At a depth of 15 layers.


In [26]:

# 3 Config

def get_depth():
    return 16

# 6 control

if __name__=="__main__":
    print("Withdrawn... was going to take 10 minutes")

Withdrawn... was going to take 10 minutes


In [22]:
def test_count_nodes():
    depth = 0
    node_count = count_nodes(depth)    
    assert (node_count == 0), 'inaccurate node count'
    
    depth = 1
    node_count = count_nodes(depth)
    assert (node_count == 1), 'inaccurate node count'
    
    depth = 2
    node_count = count_nodes(depth)
    assert (node_count == 3), 'inaccurate node count'
    
    depth = 10
    node_count = count_nodes(depth)
    assert (node_count == 1023), 'inaccurate node count'
    
    depth = 21
    node_count = count_nodes(depth)
    assert (node_count == 2097151), 'inaccurate node count'
    
def PythonSternBrocotTests():
    test_count_nodes()
    print("passed all tests")
    
PythonSternBrocotTests()

passed all tests
